In [1]:
!pip install redis
!pip install pymysql
!mysql -utdr -ptdr tdr < db.sql

     |████████████████████████████████| 71kB 2.3MB/s eta 0:00:01
     |████████████████████████████████| 51kB 1.6MB/s eta 0:00:01
mysql: [Warning] Using a password on the command line interface can be insecure.


In [3]:
import redis, json, time, pymysql
DB_Redis = redis.Redis(host='localhost', port=6379, db=0)
DB_MYSQL = pymysql.connect("localhost","tdr","tdr", "tdr")

def empty_mysql():
    cur = DB_MYSQL.cursor()
    cur.execute('DELETE FROM films WHERE 1=1')
    DB_MYSQL.commit()
    cur.close()

def empty_redis(table = ""):
    if not table.strip():
        print("plop")
        DB_Redis.delete("*")
    else:
        DB_Redis.delete("{}:*".format(table))
    
cur = DB_MYSQL.cursor()
cur.execute("INSERT INTO films (title,year,genre_list,cast_list) VALUES('foo', 2019, 'bar', 'baz')")
DB_MYSQL.commit()
cur.close()
cur = DB_MYSQL.cursor()
cur.execute('SELECT title,year,genre_list,cast_list FROM films')
print(cur.fetchall())
cur.close()
empty_mysql()
print(DB_Redis.set('foo','bar'))
print(DB_Redis.get('foo'))
print(DB_Redis.delete('foo'))
print(DB_Redis.get('foo'))

(('foo', 2019, 'bar', 'baz'),)
True
b'bar'
1
None


In [4]:
with open('movies.json') as f:
    datas_json = json.load(f)
print(datas_json[0])

{'title': 'After Dark in Central Park', 'year': 1900, 'cast': [], 'genres': []}


In [5]:
empty_redis()
t = time.time()
for film in datas_json:
    DB_Redis.set('films:{}'.format(film['title']), json.dumps(film))
t = time.time() - t
print('Insertions de {} films dans Redis : {:0.03f} secondes'.format(len(datas_json), t))

print('{} films dans Redis'.format(len(DB_Redis.keys('films:*'))))
#avatar = json.loads(DB_Redis.get('Avatar'))
#print(avatar['year'])

plop
Insertions de 28795 films dans Redis : 2.254 secondes
26791 films dans Redis


In [6]:
def mysql_insert_update(sql,values=None):
    try:
        # create a new cursor
        cur = DB_MYSQL.cursor()
        if values is not None:
            cur.execute(sql)
        elif isinstance(values, list):      
            # execute the INSERT statement
            cur.executemany(sql,values)
        else:
            cur.execute(sql,value)
        # commit the changes to the database
        DB_MYSQL.commit()
        # close communication with the database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

In [13]:
empty_mysql()
datas_tuple = []
for film in datas_json:
    datas_tuple.append((film['title'],str(film['year']),json.dumps(film['genres']),json.dumps(film['cast'])))
print(datas_tuple[0])
t = time.time()
mysql_insert_update("INSERT INTO films(title,year,genre_list,cast_list) VALUES(%s,%s,%s,%s)", datas_tuple)
t = time.time() - t
print('Insertions de {} films dans MySQL : {:0.03f} secondes'.format(len(datas_json), t))

cur = DB_MYSQL.cursor()
cur.execute('SELECT title,year,genre_list,cast_list FROM films')
print('{} films dans MySQL'.format(cur.rowcount))
cur.close()

('After Dark in Central Park', '1900', '[]', '[]')


NameError: name 'psycopg2' is not defined

In [7]:
#Création de la base de données optimisée pour la recherche des castings avec MySQL
!mysql -utdr -ptdr tdr < db2.sql
films = []
casts = []
for film in datas_json:
    films.append((film['title'],str(film['year']),json.dumps(film['genres'])))
    casts[film['title']] = film['cast']


mysql: [Warning] Using a password on the command line interface can be insecure.
^C
^C -- query aborted
ERROR 1317 (70100) at line 2: Query execution was interrupted


In [18]:
#Version optimisés pour la recherche des castings en NoSQL
t = time.time()
for film in datas_json:
    DB_Redis.set('casts:{}'.format(film['title']), json.dumps(film['cast']))
t = time.time() - t
print('Insertions de {} films dans Redis : {:0.03f} secondes'.format(len(datas_json), t))
print(json.loads(DB_Redis.get('Avatar.cast')))

Insertions de 28795 films dans Redis : 1.635 secondes
['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Michelle Rodriguez', 'Stephen Lang', 'Laz Alonso', 'Giovanni Ribisi', 'Joel Moore', 'Wes Studi']
